In [1]:
import sys
sys.path.append("..")

In [2]:
import bytelevel

In [3]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import numpy as np
from numpy.linalg import norm
import pickle

In [4]:
from keras.layers import Embedding, Conv1D, Input, GRU, LSTM, Bidirectional, Dense, UpSampling1D, Dropout, TimeDistributed, RepeatVector
from keras.models import Model

Using TensorFlow backend.


In [5]:
from hyper_params import *
import text_encoder as te
import text_decoder as td
from data_set import *

In [6]:
with open('../data/norvig/big.txt') as f:
    data = f.read()

In [7]:
chunks = make_chunks(data)

In [8]:
r = np.random.RandomState(42)

In [9]:
r.shuffle(chunks)
train_text, test_text = make_train_test(chunks)

In [10]:
embed_h = EmbeddingHyper(256, 64)
conv_h = ConvHyper(128, 6, 4)
rnn_h = RnnHyper(256, is_lstm=False, is_bidirectional=True, return_sequences=False)
encoder_h = te.Hyper(embed_h, [conv_h, rnn_h])

In [11]:
dernn_h = RnnHyper(64, is_lstm=False, is_bidirectional=False, return_sequences=True, unroll=True)
dec_h = DeconvHyper(128, 6, 4)
decoder_h = td.Hyper(256, [dernn_h, dec_h])

In [12]:
encoder = encoder_h.make_layer()
decoder = decoder_h.make_layer()

In [13]:
def make_model(max_len):
    x = Input(shape=(max_len,), name='text_input')
    h = encoder(x)
    h = decoder(h, max_len)
    model = Model(x, h)
    model.compile(optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'])

    return model

In [14]:
model64, model128, model256 = make_model(64), make_model(128), make_model(256)

In [15]:
models = {64: model64, 128: model128, 256: model256 }

In [16]:
def training_round(max_len, history=None):
    if history is None:
        initial_epoch = 0
    else:
        initial_epoch = len(history['loss'])
    train, test = SlicedData.Random(train_text, test_text, max_len, 10000, r)
    model = models[max_len]
    newhistory = model.fit(x=train.x, y=train.y,
                            epochs=initial_epoch+5, batch_size=100,
                            validation_data=(test.x, test.y),
                            initial_epoch=initial_epoch)
    if history is None:
        history = newhistory.history
    else:
        history = {key:history[key] + newhistory.history[key] for key in history.keys()}
    return history

In [17]:
history = None

In [18]:
for i in range(500):
    print(i)
    history = training_round(64, history)

0
Train on 10000 samples, validate on 1000 samples
Epoch 1/5
10000/10000 [==============================] - 14s - loss: 3.4254 - categorical_accuracy: 0.1514 - val_loss: 3.0921 - val_categorical_accuracy: 0.1640
Epoch 2/5
10000/10000 [==============================] - 9s - loss: 3.1144 - categorical_accuracy: 0.1596 - val_loss: 3.0652 - val_categorical_accuracy: 0.1641
Epoch 3/5
10000/10000 [==============================] - 9s - loss: 3.1018 - categorical_accuracy: 0.1596 - val_loss: 3.0607 - val_categorical_accuracy: 0.1641
Epoch 4/5
10000/10000 [==============================] - 9s - loss: 3.0914 - categorical_accuracy: 0.1596 - val_loss: 3.0497 - val_categorical_accuracy: 0.1641
Epoch 5/5
10000/10000 [==============================] - 10s - loss: 3.0739 - categorical_accuracy: 0.1596 - val_loss: 3.0323 - val_categorical_accuracy: 0.1642
1
Train on 10000 samples, validate on 1000 samples
Epoch 6/10
10000/10000 [==============================] - 13s - loss: 3.0456 - categorical_accur

10000/10000 [==============================] - 11s - loss: 2.2764 - categorical_accuracy: 0.2742 - val_loss: 2.3016 - val_categorical_accuracy: 0.2699
Epoch 49/50
10000/10000 [==============================] - 10s - loss: 2.2594 - categorical_accuracy: 0.2789 - val_loss: 2.2907 - val_categorical_accuracy: 0.2722
Epoch 50/50
10000/10000 [==============================] - 11s - loss: 2.2372 - categorical_accuracy: 0.2847 - val_loss: 2.2719 - val_categorical_accuracy: 0.2777
10
Train on 10000 samples, validate on 1000 samples
Epoch 51/55
10000/10000 [==============================] - 9s - loss: 2.2553 - categorical_accuracy: 0.2819 - val_loss: 2.2490 - val_categorical_accuracy: 0.2870
Epoch 52/55
10000/10000 [==============================] - 10s - loss: 2.2226 - categorical_accuracy: 0.2901 - val_loss: 2.2201 - val_categorical_accuracy: 0.2939
Epoch 53/55
10000/10000 [==============================] - 12s - loss: 2.2023 - categorical_accuracy: 0.2967 - val_loss: 2.2090 - val_categorical_

10000/10000 [==============================] - 10s - loss: 1.5025 - categorical_accuracy: 0.5163 - val_loss: 1.5477 - val_categorical_accuracy: 0.5027
19
Train on 10000 samples, validate on 1000 samples
Epoch 96/100
10000/10000 [==============================] - 11s - loss: 1.5320 - categorical_accuracy: 0.5085 - val_loss: 1.5158 - val_categorical_accuracy: 0.5142
Epoch 97/100
10000/10000 [==============================] - 10s - loss: 1.5027 - categorical_accuracy: 0.5167 - val_loss: 1.5109 - val_categorical_accuracy: 0.5147
Epoch 98/100
10000/10000 [==============================] - 11s - loss: 1.4844 - categorical_accuracy: 0.5216 - val_loss: 1.5053 - val_categorical_accuracy: 0.5153
Epoch 99/100
10000/10000 [==============================] - 10s - loss: 1.4690 - categorical_accuracy: 0.5268 - val_loss: 1.4881 - val_categorical_accuracy: 0.5202
Epoch 100/100
10000/10000 [==============================] - 11s - loss: 1.4530 - categorical_accuracy: 0.5313 - val_loss: 1.4822 - val_categ

10000/10000 [==============================] - 12s - loss: 1.2044 - categorical_accuracy: 0.6090 - val_loss: 1.2117 - val_categorical_accuracy: 0.6070
Epoch 142/145
10000/10000 [==============================] - 10s - loss: 1.1758 - categorical_accuracy: 0.6179 - val_loss: 1.2142 - val_categorical_accuracy: 0.6058
Epoch 143/145
10000/10000 [==============================] - 10s - loss: 1.1593 - categorical_accuracy: 0.6226 - val_loss: 1.1945 - val_categorical_accuracy: 0.6122
Epoch 144/145
10000/10000 [==============================] - 10s - loss: 1.1494 - categorical_accuracy: 0.6261 - val_loss: 1.1997 - val_categorical_accuracy: 0.6105
Epoch 145/145
10000/10000 [==============================] - 11s - loss: 1.1357 - categorical_accuracy: 0.6299 - val_loss: 1.1882 - val_categorical_accuracy: 0.6134
29
Train on 10000 samples, validate on 1000 samples
Epoch 146/150
10000/10000 [==============================] - 12s - loss: 1.1781 - categorical_accuracy: 0.6174 - val_loss: 1.1695 - val_c

10000/10000 [==============================] - 12s - loss: 0.9839 - categorical_accuracy: 0.6813 - val_loss: 1.0053 - val_categorical_accuracy: 0.6747
Epoch 188/190
10000/10000 [==============================] - 10s - loss: 0.9701 - categorical_accuracy: 0.6859 - val_loss: 1.0048 - val_categorical_accuracy: 0.6746
Epoch 189/190
10000/10000 [==============================] - 11s - loss: 0.9606 - categorical_accuracy: 0.6882 - val_loss: 1.0001 - val_categorical_accuracy: 0.6763
Epoch 190/190
10000/10000 [==============================] - 10s - loss: 0.9453 - categorical_accuracy: 0.6934 - val_loss: 0.9976 - val_categorical_accuracy: 0.6782
38
Train on 10000 samples, validate on 1000 samples
Epoch 191/195
10000/10000 [==============================] - 10s - loss: 0.9971 - categorical_accuracy: 0.6769 - val_loss: 0.9869 - val_categorical_accuracy: 0.6784
Epoch 192/195
10000/10000 [==============================] - 11s - loss: 0.9681 - categorical_accuracy: 0.6860 - val_loss: 0.9770 - val_c

...

10000/10000 [==============================] - 11s - loss: 0.1463 - categorical_accuracy: 0.9596 - val_loss: 0.1571 - val_categorical_accuracy: 0.9559
Epoch 2304/2305
10000/10000 [==============================] - 11s - loss: 0.1415 - categorical_accuracy: 0.9612 - val_loss: 0.1542 - val_categorical_accuracy: 0.9564
Epoch 2305/2305
10000/10000 [==============================] - 10s - loss: 0.1387 - categorical_accuracy: 0.9622 - val_loss: 0.1521 - val_categorical_accuracy: 0.9572
461
Train on 10000 samples, validate on 1000 samples
Epoch 2306/2310
10000/10000 [==============================] - 10s - loss: 0.1696 - categorical_accuracy: 0.9518 - val_loss: 0.1665 - val_categorical_accuracy: 0.9530
Epoch 2307/2310
10000/10000 [==============================] - 13s - loss: 0.1557 - categorical_accuracy: 0.9564 - val_loss: 0.1643 - val_categorical_accuracy: 0.9533
Epoch 2308/2310
10000/10000 [==============================] - 12s - loss: 0.1578 - categorical_accuracy: 0.9557 - val_loss: 0.1

10000/10000 [==============================] - 10s - loss: 0.1457 - categorical_accuracy: 0.9594 - val_loss: 0.1680 - val_categorical_accuracy: 0.9520
Epoch 2350/2350
10000/10000 [==============================] - 11s - loss: 0.1442 - categorical_accuracy: 0.9599 - val_loss: 0.1637 - val_categorical_accuracy: 0.9535
470
Train on 10000 samples, validate on 1000 samples
Epoch 2351/2355
10000/10000 [==============================] - 10s - loss: 0.3418 - categorical_accuracy: 0.9092 - val_loss: 0.2182 - val_categorical_accuracy: 0.9355
Epoch 2352/2355
10000/10000 [==============================] - 11s - loss: 0.1714 - categorical_accuracy: 0.9507 - val_loss: 0.1632 - val_categorical_accuracy: 0.9541
Epoch 2353/2355
10000/10000 [==============================] - 10s - loss: 0.1470 - categorical_accuracy: 0.9592 - val_loss: 0.1570 - val_categorical_accuracy: 0.9558
Epoch 2354/2355
10000/10000 [==============================] - 10s - loss: 0.1401 - categorical_accuracy: 0.9616 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1352 - categorical_accuracy: 0.9633 - val_loss: 0.1581 - val_categorical_accuracy: 0.9543
479
Train on 10000 samples, validate on 1000 samples
Epoch 2396/2400
10000/10000 [==============================] - 12s - loss: 0.1678 - categorical_accuracy: 0.9523 - val_loss: 0.1683 - val_categorical_accuracy: 0.9521
Epoch 2397/2400
10000/10000 [==============================] - 10s - loss: 0.1543 - categorical_accuracy: 0.9567 - val_loss: 0.1688 - val_categorical_accuracy: 0.9528
Epoch 2398/2400
10000/10000 [==============================] - 11s - loss: 0.1490 - categorical_accuracy: 0.9583 - val_loss: 0.1657 - val_categorical_accuracy: 0.9533
Epoch 2399/2400
10000/10000 [==============================] - 10s - loss: 0.1443 - categorical_accuracy: 0.9599 - val_loss: 0.1631 - val_categorical_accuracy: 0.9543
Epoch 2400/2400
10000/10000 [==============================] - 11s - loss: 0.1396 - categorical_accuracy: 0.9615 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1848 - categorical_accuracy: 0.9478 - val_loss: 0.3191 - val_categorical_accuracy: 0.9126
Epoch 2442/2445
10000/10000 [==============================] - 10s - loss: 0.3128 - categorical_accuracy: 0.9147 - val_loss: 0.1946 - val_categorical_accuracy: 0.9442
Epoch 2443/2445
10000/10000 [==============================] - 11s - loss: 0.1581 - categorical_accuracy: 0.9550 - val_loss: 0.1642 - val_categorical_accuracy: 0.9541
Epoch 2444/2445
10000/10000 [==============================] - 10s - loss: 0.1412 - categorical_accuracy: 0.9609 - val_loss: 0.1593 - val_categorical_accuracy: 0.9557
Epoch 2445/2445
10000/10000 [==============================] - 10s - loss: 0.1358 - categorical_accuracy: 0.9629 - val_loss: 0.1581 - val_categorical_accuracy: 0.9561
489
Train on 10000 samples, validate on 1000 samples
Epoch 2446/2450
10000/10000 [==============================] - 11s - loss: 0.1635 - categorical_accuracy: 0.9536 - val_loss: 0.1

10000/10000 [==============================] - 11s - loss: 0.1603 - categorical_accuracy: 0.9544 - val_loss: 0.1641 - val_categorical_accuracy: 0.9542
Epoch 2488/2490
10000/10000 [==============================] - 9s - loss: 0.1470 - categorical_accuracy: 0.9591 - val_loss: 0.1635 - val_categorical_accuracy: 0.9541
Epoch 2489/2490
10000/10000 [==============================] - 10s - loss: 0.1414 - categorical_accuracy: 0.9610 - val_loss: 0.1601 - val_categorical_accuracy: 0.9552
Epoch 2490/2490
10000/10000 [==============================] - 11s - loss: 0.1388 - categorical_accuracy: 0.9618 - val_loss: 0.1598 - val_categorical_accuracy: 0.9549
498
Train on 10000 samples, validate on 1000 samples
Epoch 2491/2495
10000/10000 [==============================] - 11s - loss: 0.2565 - categorical_accuracy: 0.9303 - val_loss: 0.3049 - val_categorical_accuracy: 0.9112
Epoch 2492/2495
10000/10000 [==============================] - 9s - loss: 0.1890 - categorical_accuracy: 0.9444 - val_loss: 0.167

In [19]:
train, test = SlicedData.Random(train_text, test_text, 64, 10000, r)
test.text[:4]

['ch\nprivateers which, in spite of the disasters of the Nile and T',
 ' things into your head! Even if\nwe put them into the wing, the m',
 'ment which he agreed to\nsend to his father-in-law, had arranged ',
 'without any definite opinion on the\nsubject of the war and witho']

In [20]:
bytelevel.prediction2str(model64.predict(test.x[:4]))

['ch\nprivateers which, in spite of the disasters of the Nile and  ',
 ' things into your head! Even if\nwe put them into the wing, the  ',
 'ment which he agreed to\nsend to his father-in-law, had arrange  ',
 'without any definite opinion on the\nsubject of the war and with ']

In [106]:
model64.save('../models/model64.h5')
with open('../models/model64.pkl', 'wb') as f:
    pickle.dump([])